In [8]:
# # EXAMPLE COMMAND: from folder examples/open_deep_research, run: python run_gaia.py --concurrency 32 --run-name generate-traces-03-apr-noplanning --model-id gpt-4o
# import argparse
# import json
# import os
# import threading
# from datetime import datetime
# from pathlib import Path
# from typing import Any
# import shutil

# import datasets
# import pandas as pd
# from dotenv import load_dotenv
# from huggingface_hub import login, snapshot_download
# from tools.reformulator import prepare_response
# from tools.run_agents import (
#     get_single_file_description,
#     get_zip_description,
# )
# from tools.text_inspector_tool import TextInspectorTool
# from tools.text_web_browser import (
#     ArchiveSearchTool,
#     FinderTool,
#     FindNextTool,
#     PageDownTool,
#     PageUpTool,
#     SimpleTextBrowser,
#     VisitTool,
# )
# from tools.visual_qa import VisualQATool
# from tqdm import tqdm

# from smolagents import (
#     CodeAgent,
#     GoogleSearchTool,
#     LiteLLMModel,
#     Model,
#     ToolCallingAgent,
# )
# from tools.agent_with_tools import create_plan_and_execute_agent, PlanExecute
# from langchain.chat_models import init_chat_model
# from tools.text_inspector_tool import TextInspectorTool
# from tools.visual_qa import VisualQATool

# # load_dotenv(override=True)
# login(os.getenv("HF_TOKEN"))

# append_answer_lock = threading.Lock()
# import sys
# sys.argv = ['']  # Override sys.argv to prevent argparse from processing notebook arguments

# import argparse


# import requests
# from langchain.tools import tool
# from dotenv import load_dotenv
# import os
# from langchain.chat_models import init_chat_model
# from langchain_community.utilities import GoogleSerperAPIWrapper
# from langchain_core.tools import Tool
# from langgraph.prebuilt import create_react_agent
# from langchain_community.tools import BraveSearch
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_community.utilities import SerpAPIWrapper
# from langchain.agents import Tool, initialize_agent, AgentType
# from langchain.chat_models import ChatOpenAI
# from tools.text_web_browser import (
#     ArchiveSearchTool,
#     FinderTool,
#     FindNextTool,
#     PageDownTool,
#     PageUpTool,
#     SimpleTextBrowser,
#     VisitTool,
# )
# from tools.text_inspector_tool import TextInspectorTool
# from tools.visual_qa import VisualQATool
# from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchResults
# import pandas as pd

# load_dotenv()

# EXAMPLE COMMAND: from folder examples/open_deep_research, run:
# python run_gaia.py --concurrency 32 --run-name generate-traces-03-apr-noplanning --model-id gpt-4o

import os
import sys
import threading
import argparse
from datetime import datetime
from pathlib import Path
import shutil
import json
import requests

import pandas as pd
import datasets
from tqdm import tqdm
from dotenv import load_dotenv

from huggingface_hub import login, snapshot_download

from tools.reformulator import prepare_response
from tools.run_agents import get_single_file_description, get_zip_description
from tools.text_inspector_tool import TextInspectorTool
from tools.text_web_browser import (
    ArchiveSearchTool,
    FinderTool,
    FindNextTool,
    PageDownTool,
    PageUpTool,
    SimpleTextBrowser,
    VisitTool,
)
from tools.visual_qa import VisualQATool
from tools.agent_with_tools import create_plan_and_execute_agent, PlanExecute

from smolagents import (
    CodeAgent,
    GoogleSearchTool,  # make sure this is distinct from Serp/BSerp tools
    LiteLLMModel,
    Model,
    ToolCallingAgent,
)

from langchain.chat_models import init_chat_model, ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
from langchain_community.utilities import GoogleSerperAPIWrapper, SerpAPIWrapper
from langchain_community.tools import BraveSearch
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchResults
from langgraph.prebuilt import create_react_agent

# Setup
load_dotenv()
login(os.getenv("HF_TOKEN"))

append_answer_lock = threading.Lock()


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
# Testing AI Agent on sample questions

question = "when is aymar de bergeyck born?"

config = {"recursion_limit": 50}
graph = create_plan_and_execute_agent(
    llm_name_planner="gpt-4.1-mini",
    llm_name_executor="gpt-4.1-mini",
    llm_name_replanner="gpt-4.1-mini",
    llm_name_answer="gpt-4.1-mini")

initial_state = PlanExecute(
    question=question,
    plan=[],
    intermediate_responses=[],
    response="",
    current_step=0,
    error_count=0,
    validation=None,
    agent_finished=False
)

workflow = initial_state
workflow =  graph.invoke(initial_state, config=config)


Agent executed step: {'messages': [SystemMessage(content='\n            You are a helpful agent that should solve the instruct given to you by the user. You have multiple tools available to you. \n\n ', additional_kwargs={}, response_metadata={}, id='9a2e477a-815b-47be-9a1c-35b62a337f5f'), HumanMessage(content='\n            The instruction is only a single step of a plan that solves a bigger problem. You should not return the final answer.\n\n            Your task is only to answer to the instruction given to you. You shouldn\'t add other text than the answer.\n\n            The final goal of the plan is to solve the following question: \n\n            when is aymar de bergeyck born?\n\n            You need to solve this instruction: Search the web for "Aymar de Bergeyck birth date" to find reliable sources that provide his date of birth.\n\n            ', additional_kwargs={}, response_metadata={}, id='1bbf5cc7-dfbe-4799-ac4c-852486780214'), AIMessage(content='', additional_kwargs={'

In [5]:
print("Final Response:", workflow)

Final Response: {'question': 'when is aymar de bergeyck born?', 'plan': ['Search the web for "Aymar de Bergeyck birth date" to find reliable sources that provide his date of birth.'], 'intermediate_responses': ['Aymar de Bergeyck was born on April 22, 1944.'], 'response': 'FINAL ANSWER: April 22 1944', 'current_step': 1, 'error_count': 0, 'validation': None, 'agent_finished': True}


In [ ]:

### IMPORTANT: EVALUATION SWITCHES

# custom_role_conversions = {"tool-call": "assistant", "tool-response": "user"}

# user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0"

# BROWSER_CONFIG = {
#     "viewport_size": 1024 * 5,
#     "downloads_folder": "downloads_folder",
#     "request_kwargs": {
#         "headers": {"User-Agent": user_agent},
#         "timeout": 300,
#     },
#     "serpapi_key": os.getenv("SERPAPI_API_KEY"),
# }

# os.makedirs(f"./{BROWSER_CONFIG['downloads_folder']}", exist_ok=True)
# print('created new folder')

def load_gaia_dataset(use_raw_dataset: bool, set_to_run: str) -> datasets.Dataset:
    if not os.path.exists("data/gaia"):
        print('gaia dataset path doesnt exist')
        if use_raw_dataset:
            print('in use raw dataset')
            snapshot_download(
                repo_id="gaia-benchmark/GAIA",
                repo_type="dataset",
                local_dir="data/gaia",
                ignore_patterns=[".gitattributes", "README.md"],
            )
        else:
            # WARNING: this dataset is gated: make sure you visit the repo to require access.
            snapshot_download(
                repo_id="smolagents/GAIA-annotated",
                repo_type="dataset",
                local_dir="data/gaia",
                ignore_patterns=[".gitattributes", "README.md"],
            )

    def preprocess_file_paths(row):
        if len(row["file_name"]) > 0:
            row["file_name"] = f"data/gaia/{set_to_run}/" + row["file_name"]
        return row
    
    eval_ds = datasets.load_dataset(
        "data/gaia/GAIA.py",
        name="2023_all",
        split=set_to_run,
    )

    eval_ds = eval_ds.rename_columns({"Question": "question", "Final answer": "true_answer", "Level": "task"})
    eval_ds = eval_ds.map(preprocess_file_paths)
    return eval_ds

def append_answer(entry: dict, jsonl_file: str) -> None:
    jsonl_path = Path(jsonl_file)
    jsonl_path.parent.mkdir(parents=True, exist_ok=True)
    with append_answer_lock, open(jsonl_file, "a", encoding="utf-8") as fp:
        fp.write(json.dumps(entry) + "\n")
    assert jsonl_path.exists(), "File not found!"
    print("Answer exported to file:", jsonl_path.resolve())

def answer_single_question_V2(
    example: dict, answers_file: str
) -> None:
    print('\n\n')
    print('================================================================================================================')
    print('example', example)

    question = example["question"]
    document_inspection_tool = TextInspectorTool(
            model=init_chat_model("gpt-4.1", model_provider="openai", temperature=0), 
            text_limit=10000)
    visual_inspection_tool = VisualQATool(model=init_chat_model("gpt-4.1", model_provider="openai", temperature=0))

    if example["file_name"]:
        if ".zip" in example["file_name"]:
            prompt_use_files = "\n\nTo solve the task above, you will have to use these attached files:\n"
            prompt_use_files += get_zip_description(
                example["file_name"], example["question"], visual_inspection_tool, document_inspection_tool
            )
        else:
            prompt_use_files = "\n\nTo solve the task above, you will have to use this attached file:\n"
            prompt_use_files += get_single_file_description(
                example["file_name"], example["question"], visual_inspection_tool, document_inspection_tool
            )

    
        # if ".zip" in example["file_name"]:
        #     folder_path = example["file_name"].replace(".zip", "")
        #     os.makedirs(folder_path, exist_ok=True)
        #     shutil.unpack_archive(example["file_name"], folder_path)
        #     string_w_paths = ""
        #     for root, dirs, files in os.walk(folder_path):
        #         print('root', root)
        #         print('dirs', dirs)
        #         print('files', files)
        #         for file in files:
        #             file_path = os.path.join(root, file)
        #             string_w_paths+= file_path+"\n"
        #     prompt_use_files = f"\n\nTo solve the question above, you will have to use these attached files: {string_w_paths}\n" 

        # else:
        #     prompt_use_files = f"\n\nTo solve the question above, you will have to use these attached files: {example["file_name"]}\n" 
        question += prompt_use_files
    print(question)
    
    # CREATE AGENT
    config = {"recursion_limit": 50}
    graph = create_plan_and_execute_agent(
        llm_name_planner="gpt-4.1-mini",
        llm_name_executor="gpt-4.1-mini",
        llm_name_replanner="gpt-4.1-mini",
        llm_name_answer="gpt-4.1-mini")

    initial_state = PlanExecute(
        question=question,
        plan=[],
        intermediate_responses=[],
        response="",
        current_step=0,
        error_count=0,
        validation=None,
        agent_finished=False
    )

    start_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    try: 
        # Run Agent
        print('Question:', question)
        workflow = initial_state
        workflow =  graph.invoke(initial_state, config=config)
        # print('QUESTION')
        # print(workflow["question"])
        # print("STEPS")
        # previous_steps_with_answers = "\n".join(
        #     f"{i}. INSTRUCTION: {workflow['plan'][i]} ANSWER: {workflow['intermediate_responses'][i]}\n" for i in range(len(workflow['intermediate_responses']))
        # )
        # print(previous_steps_with_answers)
        # print("Current step: ", workflow["current_step"])
        # print("RESPONSE")
        # print(workflow["response"])
        # print('TRUE RESPONSE')
        # print(example['true_answer'])
        # response = "No response was found"
        # for event in graph.stream(initial_state, config=config):
        #     if "planner" in event:
        #         print('PLAN')
        #         for i, p in enumerate(event['planner']['plan']):
        #             print(f"Step {i}: {p}")
        #     elif "agent" in event:
        #         print('AGENT')
        #         print(event['agent']['intermediate_responses'])
        #     elif "replan" in event:
        #         if 'plan' in event['replan']:
        #             print('REPLAN')
        #             for i, p in enumerate(event['replan']['plan']):
        #                 print(f"Step {i}: {p}")
        #         else:
        #             print('REPLAN ERROR')
        #             print(event['replan'])
        #     elif 'answer' in event:
        #         response = event['answer']['response']
        #         print('RESPONSE')
        #         print(response)
        #         break
        # workflow = {}

        raised_exception = False
    except Exception as e: 
        print("Error on", question, e)
        exception = e
        raised_exception = True

    end_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    annotated_example = {
        "question": example["question"],
        "augmented_question": question,
        "response": workflow["response"].replace('FINAL ANSWER: ', ''),
        "plan": workflow["plan"],
        "intermediate_responses": workflow["intermediate_responses"],
        "current_step": workflow["current_step"],
        "plan_with_answers": "\n".join(
            f"{i}. INSTRUCTION: {workflow['plan'][i]} ANSWER: {workflow['intermediate_responses'][i]}\n" for i in range(len(workflow['intermediate_responses']))
        ),
        "agent_error": str(exception) if raised_exception else None,
        "task": example["task"],
        "task_id": example["task_id"],
        "true_answer": example["true_answer"],
        "start_time": start_time,
        "end_time": end_time,
    }
    print('-------------------------------------------------------------')
    list_print = ['question', 'augmented_question', 'plan_with_answers', 'response', 'true_answer']
    for k in list_print:
        print(k)
        print(annotated_example[k])
    print('-------------------------------------------------------------')
    # print('annotated_example')
    # print(annotated_example)
    append_answer(annotated_example, answers_file)

def get_examples_to_answer(answers_file: str, eval_ds: datasets.Dataset) -> list[dict]:
    print(f"Loading answers from {answers_file}...")
    try:
        done_questions = pd.read_json(answers_file, lines=True)["question"].tolist()
        print(f"Found {len(done_questions)} previous results!")
    except Exception as e:
        print("Error when loading records: ", e)
        print("No usable records! ▶️ Starting new.")
        done_questions = []
    return [line for line in eval_ds.to_list() if line["question"] not in done_questions and line["file_name"]]

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--concurrency", type=int, default=8)
    parser.add_argument("--run-name", type=str, default="test1")
    parser.add_argument("--set-to-run", type=str, default="validation")
    parser.add_argument("--use-raw-dataset", action="store_true")
    return parser.parse_args()

def main():
    print('In main()')
    args = parse_args()
    print(f"Starting run with arguments: {args}")

    eval_ds = load_gaia_dataset(args.use_raw_dataset, args.set_to_run)
    print("Loaded evaluation dataset:")
    print(pd.DataFrame(eval_ds)["task"].value_counts())

    answers_file = f"output/{args.set_to_run}/{args.run_name}.jsonl"
    tasks_to_run = get_examples_to_answer(answers_file, eval_ds)
    print('answer file', answers_file)
    # Sequential processing with tqdm progress bar

    # example =  {'task_id': '5b2a14e8-6e59-479c-80e3-4696e8980152', 'question': 'The brand that makes these harnesses the dogs are wearing in the attached pic shares stories from their ambassadors on their website. What meat is mentioned in the story added Dec 8th 2022?', 'task': '3', 'true_answer': 'bacon', 'file_name': 'data/gaia/validation/5b2a14e8-6e59-479c-80e3-4696e8980152.jpg', 'file_path': '/Users/aymar/Documents/Perso_projects/Agents-course/unit4-basic-agent/tools/data/gaia/2023/validation/5b2a14e8-6e59-479c-80e3-4696e8980152.jpg', 'Annotator Metadata': {'Steps': '1. Use image search for "dog harness brands with yellow logos"\n2. Look at harnesses until a similar harness shows up\n3. Click through to see the harness\n4. Search "ruffwear"\n5. Go to the website\n6. Navigate to stories\n7. Find the story posted Dec 8th 2022\n8. Read the story to find any meats mentioned', 'Number of steps': '8', 'How long did this take?': '15 minutes', 'Tools': '1. image recognition tools\n2. image search tools\n3. web browser\n4. search engine', 'Number of tools': '4'}}
    # answer_single_question_V2(example, answers_file)

    for example in tqdm(tasks_to_run, desc="Processing tasks"):
        answer_single_question_V2(example, answers_file)

    print("All tasks processed.")
print('beforemain')
main()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Make sure you deactivated any VPN like Tailscale, else some URLs will be blocked!
created new folder
beforemain
In main()
Starting run with arguments: Namespace(concurrency=8, model_id='o1', run_name='test1', set_to_run='validation', use_open_models=False, use_raw_dataset=False)
gaia dataset path doesnt exist


Fetching 161 files:   2%|▏         | 3/161 [00:05<04:23,  1.67s/it]


KeyboardInterrupt: 